<a href="https://colab.research.google.com/github/chemicoPy/ML-Forex-Market-Prediction/blob/Neural-Prophet-Network/Neural_Prophet_Network_on_stock_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
# drive.mount("/content/gdrive")

In [ ]:
!pip install python-binance
!pip install neuralprophet
!pip install neuralprophet[live]

     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 288 kB 17.6 MB/s 
     |████████████████████████████████| 214 kB 56.9 MB/s 
     |████████████████████████████████| 103 kB 56.1 MB/s 
     |████████████████████████████████| 1.3 MB 49.1 MB/s 
     |████████████████████████████████| 271 kB 47.9 MB/s 
     |████████████████████████████████| 160 kB 52.1 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 155 kB 14.9 MB/s 
  Attempting uninstall: holidays
    Found existing installation: holidays 0.10.5.2
    Uninstalling holidays-0.10.5.2:
      Successfully uninstalled holidays-0.10.5.2


In [ ]:
import pandas as pd
import numpy as np
from binance.client import Client
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
import time
import math
import os
import io
from IPython.display import clear_output
from datetime import date, timedelta
from neuralprophet import NeuralProphet
import csv
import warnings

In [ ]:
# dir_link=input('/content/drive')
# file_name='/binance_data.csv'
# compl_link=dir_link+file_name
# compl_link

# OR if i'm to read csv from local

""" from google.colab import files
uploaded = files.upload() """


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:

def data():
    API_KEY = 'cmC20L6OdOe6nfS4iD0T3aIYYbGh53G9nsz1kSO4pd5JsVtVbsycpvzxIwN89AEN'
    API_SECRET = 'JjyurQGV0hcbsEhsdUQoaoaeA85kw33xRrEJywhOJfI6ZNUvT42RMkQK1ed59gyh'
    #API_KEY = 'FiehiL4oASF4FDsR5oAUfBqhdOjR4vwfwQltHbX4ucdr78XY6nA8bKsVorwfMMpt'
    #API_SECRET = 'k4pkBSFEjupQN63PYYBDl3mgKrLAMEopKCcy8YcLusi0mBrxyxyCLbQB12awL3QC'
    # API_KEY = 'aTrEpYSztGNxooC07JI0mEfFJUud6jg2yOJkhMt9xC5IQBFbq19n5SApaG0eeHoS'
    # API_SECRET ='c8AoWE0caEmyGuQuLQSKFPLdcD1UN0T9MBDcuRbWPYT7nlk308XWmU0YzcUwvSwq'

    client = Client(API_KEY, API_SECRET)
    whole_data = []

    all_coins = ["BNBUSDT", "ETHUSDT", "ADAUSDT", "DOTUSDT", "BTCUSDT", "FTMUSDT", "DOGEUSDT", "LINKUSDT", "SUSHIUSDT", "SOLUSDT",
                 "IOSTUSDT", "LTCUSDT", "XRPUSDT", "SXPUSDT", "MATICUSDT", "BCHUSDT"]




    td=timedelta(-10)
    time_now = datetime.now()
    start1 = time_now + td
    start = start1.strftime('%Y-%m-%d- %H:%M:%S')



    Current_Date = datetime.today()

    end=Current_Date.strftime ('%Y-%m-%d- %H:%M:%S')






    # data_of_coin = input('Enter the name of coin along with USDT such BNBUSDT')
    data_of_coin='BTCUSDT'

    index_of_coin = all_coins.index(data_of_coin)


    print(index_of_coin)
    coin = all_coins[index_of_coin]
    # print(coin)

    # interval = input('Enter 5m  for the INTERVAL of 5MINUTE \n , 15m for the INTERVAL of 15MINUTE, \n'
    #                  '3 for the INTERVAL of 30MINUTE \n, 4for the INTERVAL of 1 HOUR, \n'
    #                  ' 5 for the INTERVAL of 4HOUR \n, 8h for the INTERVAL of 8 HOUR \n')
    interval='5m'
    dictionary = {'1m': Client.KLINE_INTERVAL_1MINUTE,'5m': Client.KLINE_INTERVAL_5MINUTE, '15m': Client.KLINE_INTERVAL_15MINUTE,
                  '30m': Client.KLINE_INTERVAL_30MINUTE, '1h': Client.KLINE_INTERVAL_1HOUR,
                  '4h': Client.KLINE_INTERVAL_4HOUR, '8h': Client.KLINE_INTERVAL_8HOUR}
    # print('start',start, type(start), 'end', end, type(end) )
    for runner in range(0, 1):

        name = []
        klines = client.get_historical_klines(coin, dictionary[interval], start, end)


        # getting name of coin.
        for i in klines:
            name.append(coin)

        df = pd.DataFrame(klines)  # data of coin is prepared.
        df['Coin_Name'] = name

        whole_data.append(df)
        # print(df)

    merging_data = pd.concat(whole_data, axis=0)
    #print(merging_data)

    m_data = merging_data.drop(11, axis=1)
    global N_data
    N_data = m_data.rename(columns={0: 'Open time', 1: 'Open', 2: 'High', 3: 'Low', 4: 'Close', 5: 'Volume',
                                          6: 'Close Time', 7: 'Qoute Asset Volume', 8: 'No of Trades', 9: 'Taker buy',
                                          10: 'Taker buy quote asset volume', 'Coin_Name': 'Coin_Name'})

    o_t_c = N_data['Open time'] / 1000
    c_t_c = N_data['Close Time'] / 1000

    o_t_list = []
    for timer in o_t_c:
        timestamp = datetime.fromtimestamp(timer)
        o_t_value = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        o_t_list.append(o_t_value)
    N_data['Open time'] = o_t_list

    c_t_list = []
    for timer in c_t_c:
        timestamp = datetime.fromtimestamp(timer)
        c_t_value = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        c_t_list.append(c_t_value)
    N_data['Close Time'] = c_t_list
    N_data.to_csv(compl_link)
    return N_data




# print('Data File Created')



In [ ]:
def all_fun():
  data().to_csv(compl_link)


In [ ]:
while True:
   all_fun()
# time.sleep(292)
   clear_output(wait=True)
#500
   data().to_csv(compl_link)




In [ ]:

data = pd.read_csv(compl_link, index_col='Open time', usecols=[ 'Open time', 'Close Time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Qoute Asset Volume', 'No of Trades', 'Taker buy',
        'Taker buy quote asset volume'])

data=data[:-1]

# For case of reading csv file from local:

""" import io
data = pd.read_csv(io.BytesIO(uploaded['binance_data.csv']), index_col=[0]) """

In [ ]:
data.reset_index( inplace=True)

data

In [ ]:
"""Above is a 5-minute interval stcok data for BTCUSDT """

In [ ]:
""" Renaming columns for (Close Date & CLose Price) - Date and Close Price to "ds" & "y" respectively """ 
"""For predicting just close price as requested"""

data1 = data.copy()
data1.rename(columns = {"Close Time":"ds", "Close": "y" }, inplace = True)

In [ ]:
data1

In [ ]:
"""Make a copy of data if i'll be interested in predicting open price too """

data2 = data.copy()
data2.rename(columns = {"Open time":"ds", "Open": "y" }, inplace = True)

#data2



In [ ]:
data1.drop(data1.columns.difference(['y','ds']), 1, inplace=True)

In [ ]:
data1

In [ ]:

m = NeuralProphet(
    changepoints_range=0.95,
    n_changepoints=50,
    trend_reg=1.5,
    weekly_seasonality=False,
    daily_seasonality=10,
)
metrics = m.fit(data1, freq='5min')
future = m.make_future_dataframe(data1, periods=60//5*24*7, n_historic_predictions=len(data1))
forecast = m.predict(future)
fig = m.plot(forecast)
# fig_comp = m.plot_components(forecast)
#fig_param = m.plot_parameters()

In [ ]:
future = m.make_future_dataframe(data1, periods=50)
forecast = m.predict(future)

In [ ]:
forecast

In [ ]:
""" For Opens time (data) """

data2

In [ ]:
data2.drop(data2.columns.difference(['y','ds']), 1, inplace=True)

In [ ]:
data2

In [ ]:

model = NeuralProphet(
    changepoints_range=0.95,
    n_changepoints=50,
    trend_reg=1.5,
    weekly_seasonality=False,
    daily_seasonality=10,
    yearly_seasonality=False
)
metrics = model.fit(data2, freq='5min')
future_2 = model.make_future_dataframe(data2, periods=60//5*24*7, n_historic_predictions=len(data2))
forecast_2 = model.predict(future)
fig = model.plot(forecast_2)

#fig_comp = model.plot_components(forecast_2)
#fig_param = model.plot_parameters()

In [ ]:
future_2 = model.make_future_dataframe(data1, periods=3)
forecast_2 = model.predict(future_2)

In [ ]:
forecast_2

In [ ]:
model.plot_parameters()